In [1]:
import pickle
import random
import string

# Nuevos usuarios

Vamos a asociar un nombre de usuario y contraseña a cada uno de los userIds existentes en el dataset de ratings

In [ ]:
def create_user_password_dict(num_users):
    users = {}
    for i in range(num_users):
        user = f"user{i + 1}"
        password = ''.join(random.choices(string.ascii_letters + string.digits, k=12))
        users[i + 1] = {"user": user, "password": password}
    return users

num_users = 610
user_password_dict = create_user_password_dict(num_users)

# Show users' information
for user_id, data in user_password_dict.items():
    print(f"ID: {user_id} | User: {data['user']} | Password: {data['password']}")


In [ ]:
# Change the password of users 1 and 4
user_password_dict[1]["password"] = "123"
user_password_dict[4]["password"] = "456"

# # Show users' information
# for user_id, data in user_password.items():
#     print(f"ID: {user_id} | User: {data['user']} | Password: {data['password']}")

In [ ]:
# Save the dictionary
with open("user_password_dict.pkl", 'wb') as file:
        pickle.dump(user_password_dict, file)

Añadir nuevos usuarios a la base de datos

In [ ]:
def add_new_user(username, password, users):
    for data in users.values():
        if data["user"] == username:
            print(f"El nombre de usuario '{username}' ya está en uso.")
            return

    # Obtener el último ID existente y aumentarlo en 1
    last_id = max(users.keys())
    new_id = last_id + 1

    # Agregar el nuevo usuario al diccionario
    users[new_id] = {"user": username, "password": password}
    print(f"Nuevo usuario agregado: ID: {new_id} | User: {username} | Password: {password}")

# Ejemplo de uso
username = input("Introduce el nombre de usuario: ")
password = input("Introduce la contraseña: ")

add_new_user(username, password, user_password_dict)

In [ ]:
user_password_dict

In [ ]:
existing_usernames = [user['user'] for user in user_password_dict.values()]
existing_usernames

In [ ]:
def existing_username(username,user_password_dict):
    return username in [user['user'] for user in user_password_dict.values()]

existing_username("user5555",user_password_dict)

In [ ]:
# Load user-password database
with open('user_password_dict.pkl', 'rb') as file:
    user_password_dict = pickle.load(file)

In [ ]:
user_password_dict

## Obtener las películas con mayor número de valoraciones

In [2]:
import os
os.chdir('../movies')
from movieLens import MovieLens

ml = MovieLens()
ratings = ml.ratings.copy()

In [3]:
k = 15
top_movies = ratings.groupby('movieId').size().nlargest(k).index.tolist()
top_movies_titles = [ml.movieID_to_name.get(movie_id, 'Unknown') for movie_id in top_movies]
top_movies_titles

['Forrest Gump',
 'Shawshank Redemption, The',
 'Pulp Fiction',
 'Silence of the Lambs, The',
 'Matrix, The',
 'Star Wars: Episode IV - A New Hope',
 'Jurassic Park',
 'Braveheart',
 'Terminator 2: Judgment Day',
 "Schindler's List",
 'Fight Club',
 'Toy Story',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Usual Suspects, The',
 'American Beauty']

Vamos a adaptar el nombre de las películas para que sea más sencillo de leer

In [5]:
top_movies_titles = ['Forrest Gump',
 'The Shawshank Redemption',
 'Pulp Fiction',
 'The Silence of the Lambs',
 'The Matrix',
 'Star Wars: Episode IV - A New Hope',
 'Jurassic Park',
 'Braveheart',
 'Terminator 2: Judgment Day',
 "Schindler's List",
 'Fight Club',
 'Toy Story',
 'Star Wars: Episode V',
 'The Usual Suspects',
 'American Beauty']

In [6]:
top_movies_dict = {}
for movie_id, movie_title in zip(top_movies, top_movies_titles):
    top_movies_dict[movie_id] = movie_title
    
top_movies_dict

{356: 'Forrest Gump',
 318: 'The Shawshank Redemption',
 296: 'Pulp Fiction',
 593: 'The Silence of the Lambs',
 2571: 'The Matrix',
 260: 'Star Wars: Episode IV - A New Hope',
 480: 'Jurassic Park',
 110: 'Braveheart',
 589: 'Terminator 2: Judgment Day',
 527: "Schindler's List",
 2959: 'Fight Club',
 1: 'Toy Story',
 1196: 'Star Wars: Episode V',
 50: 'The Usual Suspects',
 2858: 'American Beauty'}

In [7]:
# Save the dictionary
with open("../dashboard/top_movies_dict.pkl", 'wb') as file:
        pickle.dump(top_movies_dict, file)

In [ ]:
for i, pelicula in enumerate(top_movies_dict.values()):
    print(i)
    print(pelicula)

In [ ]:
ratings['userId'].max()+1

In [ ]:
# Load user-password database
with open('../dashboard/valores_slider.pkl', 'rb') as file:
    valores_slider = pickle.load(file)
valores_slider

## Recomendaciones

In [ ]:
import os
os.chdir('../movies')
from movieLens import MovieLens

ml = MovieLens()
name_to_movieID = ml.name_to_movieID.copy()

with open("../dashboard/name_to_movieID.pkl", 'wb') as file:
        pickle.dump(name_to_movieID, file)

In [ ]:
name_to_movieID

In [ ]:
name_to_movieID['Toy Story']

In [ ]:
# Números de películas a mostrar
import os
print(os.getcwd)

movie_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

def get_movie_images(numbers):
    image_folder = "images/"
    movie_images = []

    for number in numbers:
        image_path = os.path.join(image_folder, f"{number}.jpg")
        print(image_path)
        if os.path.isfile(image_path):
            print("dentro")
            movie_images.append(image_path)

    return movie_images

# Obtener las rutas de las imágenes de las películas
movie_images = get_movie_images(movie_numbers)
movie_images